# Set up

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
os.chdir('C:\\Users\\User\\Desktop\\Internship')
data_path = 'data.h5'
data = h5py.File(data_path, 'r') 
import random


N = 35 # length of lagged stim vector, how many past times we want it to depend on, N can be set to equal t if we want to consider all past values 
M = 35 #lenght of EEG feedback vector 
x = np.hstack([data[f'stim/part{0}'][:]])

Y_t_allelectrodes = np.hstack([data[f'eeg/P00/part{0}'][:]]) #only looking at participant P00. but this is for all electrodes 
y = Y_t_allelectrodes[51] #update for speicific electrode  

from scipy.stats import pearsonr

'''model parameters'''
np.random.seed(0)
k = np.random.normal(0, 0.01, N) 

np.random.seed(1)
h = np.random.normal(0, 0.01, M)

# The model - definitions
Lagged stimulus input vector: (N x 1) \begin{align}
    \textbf{X} _{t} &= \begin{bmatrix}
           x_{t} \\
           x_{t-1} \\
           \vdots \\
           x_{t-N+1}
         \end{bmatrix}
  \end{align} 


EEG feedback vector (predicted EEG): (M x 1) \begin{align}
    \mathbf{\hat{Y}}_{t-1} &= \begin{bmatrix}
           \hat{y}_{t-1} \\
           \vdots \\
           \hat{y}_{t-M}
         \end{bmatrix}
 \end{align} 
  

Model parameters: \begin{align}
     \boldsymbol{k} = (k_1, k_2, ..., k_N)\\
     \boldsymbol{h} = (h_1, h_2, ..., h_M)
\end{align} 

# Getting  $\textbf{X}_{t}$ and $\mathbf{\hat{Y}}_{t-1} $ and the toy model data

y_hat: \begin{align}
\hat{y}_{t}= \boldsymbol{k} \cdot \mathbf{X}_t + \boldsymbol{h} \cdot \mathbf{\hat{Y}}_{t-1}
\end{align} 

y_toy: \begin{align}
{y}_{t}= \boldsymbol{n} \cdot \mathbf{X}_t + \boldsymbol{m} \cdot \mathbf{{Y}}_{t-1}
\end{align} 



In [ ]:
'''Generating the lagged stimulus vector'''

def get_X(x, t, N): #t is the specific time and N is the legnth of the vector
    X = np.zeros(N) #initialise X_t as a vector of zeros 
    if t >= N: 
        X = np.flip(x[t-N+1:t+1])  #if t>= N, we update X to contain the N elements preceding t from x
    if t < N:
        X[:t] = np.flip(x[:t])  #otherwise we extract the first t elements from x 
    return X



'''Generating the data for the toy model'''

np.random.seed(2)
n = np.random.normal(0, 0.1, N) 
np.random.seed(3)
m = np.random.normal(0, 0.1, M)

noise = np. random.normal(0, 0.01, x.shape)

def generate_data(datasize, stimulus, m, n, noise): 
    Y_tmin1 = np.zeros(M)
    Y = np.zeros(datasize)

    for i in range(datasize):
        y_i =0 
        Y_tmin1 = np.roll(Y_tmin1, 1)
        Y_tmin1[0]= y_i
        X_t = get_X(x, i, N)
        y_i = n @ X_t + m @ Y_tmin1
        Y[i] = y_i
    return Y + noise

#y = generate_data(20271, x, m,n, noise)


'''Generating predictions'''

batchsize = 300
iterate = int(np.ceil((2/3)*len(x)/batchsize))
first = len(x)- batchsize*iterate

def get_predictions(k, h, x, stop):
    Y_hat = np.zeros(M)
    predictions = np.zeros(stop)

    for i in range(stop):
        y_i =0 
        Y_hat = np.roll(Y_hat, 1)
        Y_hat[0]= y_i
        X_t = get_X(x, i, N)
        y_i = k @ X_t + h @ Y_hat 
        predictions[i] = y_i
    return predictions

predictions = get_predictions(k,h,x, 20271) 


def get_Y_hat(t):
    Y_hat = np.zeros(M)
    if t>= M: 
        Y_hat = np.flip(predictions[t-M : t])
    if t<M: 
        Y_hat[:t] =  np.flip(predictions[ : t])
        
    return Y_hat

# Getting errors 

get_sq_err_t: \begin{align}
\epsilon_{t}= (\hat{y}_{t}-y_{t})^{2} 
\end{align}


get_total_sq_err: \begin{align}
\epsilon = \sum_{t=start}^{stop}\epsilon_{t} 
\end{align}

avg_batch_error:  \begin{align}
\epsilon_{avg} = \frac{\epsilon}{batchsize}
\end{align}

In [ ]:
'''Getting the squared error at time t''' 
def get_sq_err_t(x, y, k, h, t): 
    Y_hat = get_Y_hat(t)
    X = get_X(x, t, N)
    y_hat = k @ X + h @ Y_hat
    squared_error = (y_hat - y[t])**2
    return squared_error 

'''summing to get total squared error'''
def get_total_sq_err(x, y, k, h, start, stop):
    total_errors = []
    for t in range(start, stop): 
        squared_error = get_sq_err_t(x, y, k, h, t)
        total_errors.append(squared_error)
    total_sq_err = np.sum(total_errors)
    return total_sq_err   

'''average squared error per batch'''
def avg_batch_error(x, y, k, h, start, stop, batchsize): 
    tot_error= get_total_sq_err(x, y, k, h, start, stop)
    avg_batch_err = tot_error/batchsize
    return(avg_batch_err)

# Derivative of error wrt k

derivative_array_k:
(M xN) \begin{align}
    \frac{\partial\mathbf{\hat{Y}}_{t-1}}{\partial{\boldsymbol{k}}} &= \begin{bmatrix}
           \frac{\partial{\hat{y}_{t-1}}}{\partial{\boldsymbol{k}}} \\
           \vdots \\
           \frac{\partial{\hat{y}_{t-M}}}{\partial{\boldsymbol{k}}}
         \end{bmatrix}
 \end{align} 

d_dk: (N x 1) \begin{align}
\frac{\partial{\hat{y}_{t}}}{\partial{k}} = \mathbf{X}_t + \boldsymbol{h} \cdot \frac {\partial \mathbf{\hat{Y}}_{t-1}}{\partial{\boldsymbol{k}}}
\end{align} 

grad_times_error_k: \begin{align}
\frac{\partial{\hat{y}_{t}}}{\partial{\boldsymbol{k}}} (\hat{y}_{t} - y_t)
\end{align} 

error_deriv_wrt_k: \begin{align}
\frac{\partial{\epsilon}}{\partial{\boldsymbol{k}}} =
2 \sum_{t=start}^{stop}\frac{\partial{\epsilon_{t}}}{\partial{\boldsymbol{k}}}
= 2 \sum_{t=start}^{stop}\frac{\partial{\hat{y}_{t}}}{\partial{\boldsymbol{k}}} (\hat{y}_{t} - y_t)
\end{align} 

In [ ]:
def err_derivative_k(x, y, t, k, h, start = 0, alpha_k = 1e-5):
    derivative_array_k = np.zeros((M,N)) #array of zeroes to fill with the derivatives (d/dk(yhat_t-1),...,d/dk(yhat_t-M))
    Y_hat = np.zeros(M) #array of zeroes to fill with the values of yhat_t-1,...,yhat_t-M
    grad_times_error_array_k = np.zeros((t-start,N))
    for i in range(start, t): 
        X = get_X(x, i, N)
        
        derivative_array_k = np.roll(derivative_array_k, 1, axis =0) #shifts all elements of the first row down one, and makes first row contain 0s
        d_dk = X + h @ derivative_array_k #formula for the derivative of y_hat at time t, basically eliminitates k this will be a vector
        derivative_array_k[0,:] = d_dk.T #replaces first row with derivatives 
        
        y_t = y[i] 
        Y_hat = get_Y_hat(t)
        y_hat = k @ X + h @ Y_hat 
    
        grad_times_error_k = d_dk * (y_hat - y_t)
        grad_times_error_array_k[i-start-1,:] = grad_times_error_k.T
    error_deriv_wrt_k = 2 * np.sum(grad_times_error_array_k, axis = 0)
        
    return error_deriv_wrt_k + alpha_k*2*k

# Derivative of error wrt h

derivative_array_h:
(MxM) \begin{align}
    \frac{\partial\mathbf{\hat{Y}}_{t-1}}{\partial{\boldsymbol{h}}} &= \begin{bmatrix}
           \frac{\partial{\hat{y}_{t-1}}}{\partial{\boldsymbol{h}}} \\
           \vdots \\
           \frac{\partial{\hat{y}_{t-M}}}{\partial{\boldsymbol{h}}}
         \end{bmatrix}
 \end{align} 

d_dh: (N x 1) \begin{align}
\frac{\partial{\hat{y}_{t}}}{\partial{\boldsymbol{h}}} = \mathbf{\hat{Y}}_{t-1}+ \boldsymbol{h} \cdot \frac {\partial \mathbf{\hat{Y}}_{t-1}}{\partial{\boldsymbol{h}}}
\end{align} 

grad_times_error_h: \begin{align}
\frac{\partial{\hat{y}_{t}}}{\partial{\boldsymbol{h}}} (\hat{y}_{t} - y_t)
\end{align} 

error_deriv_wrt_h: \begin{align}
\frac{\partial{\epsilon}}{\partial{\boldsymbol{h}}} =
2 \sum_{t=start}^{stop}\frac{\partial{\epsilon_{t}}}{\partial{\boldsymbol{h}}}
= 2 \sum_{t=start}^{stop}\frac{\partial{\hat{y}_{t}}}{\partial{\boldsymbol{h}}} (\hat{y}_{t} - y_t)
\end{align} 


In [ ]:
def err_derivative_h(x, y, t, k, h, start = 0, alpha_h= 1e-5): 
    derivative_array_h = np.zeros((M,M))
    Y_hat = np.zeros(M)
    grad_times_error_array_h = np.zeros((t-start,M))

    for i in range(start, t):
        Y_hat = get_Y_hat(t)
        derivative_array_h = np.roll(derivative_array_h, 1, axis = 0)
        d_dh = Y_hat + h @ derivative_array_h
    
        derivative_array_h[0,:] = d_dh.T
        X = get_X(x, i, N)
        y_t = y[i]
        y_hat = k @ X + h @ Y_hat
        
        grad_times_error_h = d_dh * (y_hat - y_t)
        grad_times_error_array_h[i-start-1,:] = grad_times_error_h.T
    
    error_deriv_wrt_h = 2 * np.sum(grad_times_error_array_h, axis = 0)
    return error_deriv_wrt_h + alpha_h*2*h


#err_derivative_h(40,k,h, start = 0)

# Batch Descent
Updating k and h: 
\begin{align}
\boldsymbol{k}^{(i+1)}=\boldsymbol{k}^{(i)} - \lambda_k \frac{\partial{\epsilon^{(i)}}}{\partial{\boldsymbol{k}}} 
\end{align} 


\begin{align}
\boldsymbol{h}^{(i+1)}=\boldsymbol{h}^{(i)} - \lambda_h \frac{\partial{\epsilon^{(i)}}}{\partial{\boldsymbol{h}}} 
\end{align} 

avg_batch_error: \begin{align}
\epsilon_{\text{avg}}= \frac{\epsilon}{batchsize}
\end{align} 

In [ ]:
'''simple batch descent'''

def batchDescent(x, y, k, h, lambda_k, lambda_h, batchsize =300): #err_k = total_errors_k etc, 13514/233 = 58
    MIN = 0
    
    iterate = int(np.ceil(len(x)/batchsize))
                 
    for j in range(iterate):
        MIN = j*batchsize
        if batchsize > x.size - MIN: #if the remainder of the train set < the batchsize, the final batch = remainder 
            batchsize = x.size%MIN
        #start = time.time() 
        
        
        T = MIN + batchsize
      
        err_deriv_k = err_derivative_k(x, y, T, k, h, MIN)
        err_deriv_h = err_derivative_h(x, y, T, k, h, MIN)
        
        k = k - lambda_k * err_deriv_k
        h = h - lambda_h * err_deriv_h
        
        avg_batch_err = avg_batch_error(x, y, k, h, MIN, T, batchsize)
        print(avg_batch_err)
        
        predictions = get_predictions(k,h,x, T) #update 20271 to be T
        #print(pearsonr(predictions, y)) #update y to be y[MIN:T]
              
    return k, h


k,h = batchDescent(x, y, k, h,1e-7,1e-7)

# Plotting predictions vs actual data


In [ ]:
#y = generate_data(20271, x, m, n, noise)
predictions = get_predictions(k,h,x, 20271)
times = range(20271)
plt.plot(times,y)
plt.plot(times, predictions)


In [ ]:
from scipy.stats import pearsonr
pearsonr(predictions, y)

# Alternative batch descents

In [ ]:
'''batch descent looped over training data many times'''

def loopBatchDescent(x, y, k, h, lambda_k, lambda_h, batchsize =500): #err_k = total_errors_k etc, 13514/233 = 58
    MIN = 0

    for i in range(25):
        iteration_error = []
        iterations = int(np.ceil(len(x)/batchsize))  
        #print(iterations)
        for j in range(iterations):
            MIN = j*batchsize 
            if batchsize > x.size - MIN: #if the remainder of the train set < the batchsize, the final batch = remainder 
                batchsize = x.size%MIN
        
            T = MIN + batchsize
   
            err_deriv_k = err_derivative_k(x, y, T, k, h, MIN)
            err_deriv_h = err_derivative_h(x, y, T, k, h, MIN)
        
            k = k - lambda_k * err_deriv_k
            h = h - lambda_h * err_deriv_h
            
            avg_batch_err = avg_batch_error(x, y, k, h, MIN, T, batchsize)
            iteration_error.append(avg_batch_err)
        predictions = get_predictions(k,h,x, 20271)
        print(pearsonr(predictions, y))
        
        avg_iteration_error = np.sum(iteration_error)/iterations
        print(avg_iteration_error)
        
    return k, h

k,h = loopBatchDescent(x, y, k, h, 1e-6,1e-6)

In [ ]:
'''Nesterov momentum, looped batch descent'''

def loopMomentumBatchDescent(x, y, k, h, lambda_k, lambda_h, gamma_k, gamma_h, batchsize =500): #err_k = total_errors_k etc, 13514/233 = 58
    
    for i in range(25):
        iteration_error = []
        
        MIN = 0
        update_h = 0 #(initialised)
        update_k = 0
        
        iterations = int(np.ceil(len(x)/batchsize))
         
        #print(iterations)
        for j in range(iterations):
            MIN = j*batchsize
            if batchsize > x.size - j*batchsize: #if the remainder of the train set < the batchsize, the final batch = remainder 
                batchsize = x.size%(j*batchsize)
           
            T = MIN + batchsize
    
            err_deriv_k = err_derivative_k(x, y, T, k, h, MIN)
            err_deriv_h = err_derivative_h(x, y, T, k, h, MIN)
        
            update_k = gamma_k*update_k + lambda_k* err_deriv_k
            k = k + gamma_k*update_k - lambda_k*err_deriv_k
        
            update_h = gamma_h*update_h + lambda_h* err_deriv_h
            h = h +  gamma_h*update_h - lambda_h* err_deriv_h
        
            avg_batch_err = avg_batch_error(x, y, k, h, MIN, T, batchsize)
            iteration_error.append(avg_batch_err)
            
        predictions = get_predictions(k,h,x, 20271)
        #print(pearsonr(predictions, y))
        
        avg_iteration_error = np.sum(iteration_error)/iterations
        print(avg_iteration_error)
        
    return k, h

k,h = loopMomentumBatchDescent(x, y, k, h, 1e-7,1e-7, 0.5, 0.5)